# CustomLayer 定義

如果你想要自己根據論文公式刻出一個model，下面一有一個最基本的範例可供使用
PyTorch 在定義model會需要用到Parameter來打包可訓練參數
這樣在優化器指定要優化的參數時，才可以透過Module.parameter()這個函式來快速調參

而要注意的是當資料是以Batch的形式呈現，計算時要注意batch的軸不能消失
否則在訓練資料的過程中就會失去Batch Training的意義

也就是說公式需要改寫為
## $$y=xW^T+b$$

In [1]:
import torch

c:\Users\Wen2Tee5\Desktop\Postgraduate\torchPlayGround\torchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 自定義一個layer來做影像辨識
class LinearLayer(torch.nn.Module):

    def __init__(self, in_feature, out_feature) -> None:
        super(LinearLayer, self).__init__()
        self.w1 = torch.nn.Parameter(torch.randn(size = (out_feature, in_feature), requires_grad=True))
        self.b1 = torch.nn.Parameter(torch.randn(size = (out_feature, )))

    def forward(self, x):
        x = x @ self.w1.T + self.b1
        return x

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [5]:
train_dataset = MNIST(root='./basic_dataset', train = True, download= True, transform=ToTensor())
test_dataset = MNIST(root='./basic_dataset', train = False, download= True, transform=ToTensor())

In [6]:
trainLoader = DataLoader(train_dataset, batch_size=32, shuffle= True)
testLoader = DataLoader(test_dataset, batch_size=32, shuffle= True)

In [7]:
import torch.nn as nn
import torch.optim as optim

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            LinearLayer(28*28, 512),
            nn.ReLU(),
            LinearLayer(512, 512),
            nn.ReLU(),
            LinearLayer(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): LinearLayer()
    (1): ReLU()
    (2): LinearLayer()
    (3): ReLU()
    (4): LinearLayer()
  )
)


In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainLoader, model, loss_fn, optimizer)
    test(testLoader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 4782.626953  [    0/60000]
loss: 197.373199  [ 3200/60000]
loss: 220.579132  [ 6400/60000]
loss: 82.406097  [ 9600/60000]
loss: 171.952866  [12800/60000]
loss: 404.325745  [16000/60000]
loss: 7.570625  [19200/60000]
loss: 62.055294  [22400/60000]
loss: 13.345665  [25600/60000]
loss: 179.331604  [28800/60000]
loss: 0.000000  [32000/60000]
loss: 111.966049  [35200/60000]
loss: 127.089119  [38400/60000]
loss: 65.335205  [41600/60000]
loss: 130.975403  [44800/60000]
loss: 30.974018  [48000/60000]
loss: 77.031799  [51200/60000]
loss: 17.194908  [54400/60000]
loss: 50.983852  [57600/60000]
Test Error: 
 Accuracy: 90.9%, Avg loss: 60.267712 

Epoch 2
-------------------------------
loss: 73.616791  [    0/60000]
loss: 108.506149  [ 3200/60000]
loss: 38.035004  [ 6400/60000]
loss: 173.741913  [ 9600/60000]
loss: 0.000000  [12800/60000]
loss: 73.306580  [16000/60000]
loss: 35.227268  [19200/60000]
loss: 63.249058  [22400/60000]
loss: 23.871391  [256